# ニューラルネットワークの動作確認

<a href="http://scikit-learn.org/dev/modules/generated/sklearn.neural_network.MLPClassifier.html#sklearn-neural-network-mlpclassifier"><b>sklearn.neural_network.MLPClassifier</b></a> を使用します。

多層パーセプトロン分類器（Multi-layer Perceptron classifier）とのことです。

テストデータ「test_daikin_conversation.csv」を使用し、動きを簡単に確認してみます。

## (1) テストデータ／環境準備

In [1]:
'''
    テスト環境を準備するためのモジュールを使用します。
'''
import sys
import os
learning_dir = os.path.abspath("../../") #<--- donusagi-bot/learning
os.chdir(learning_dir)

if learning_dir not in sys.path:
    sys.path.append(learning_dir)

from prototype.modules import TestTool

### (1-1) テストデータをコピー

In [2]:
'''
    データファイルは、既存の訓練データを別場所にコピーしてから使用します
    テストデータは、csv_file_name で指定した複数件のファイルを使用します。
'''
csv_file_names = [
    'test_daikin_conversation.csv',
    'test_benefitone_conversation.csv',
    'test_septeni_conversation.csv',
    'test_ptna_conversation.csv',
]
temp_path = TestTool.copy_testdata_csv(learning_dir, csv_file_names)

CSV file for test=[/Users/makmorit/GitHub/donusagi-bot/learning/prototype/resources/test_daikin_conversation.csv]
CSV file for test=[/Users/makmorit/GitHub/donusagi-bot/learning/prototype/resources/test_benefitone_conversation.csv]
CSV file for test=[/Users/makmorit/GitHub/donusagi-bot/learning/prototype/resources/test_septeni_conversation.csv]
CSV file for test=[/Users/makmorit/GitHub/donusagi-bot/learning/prototype/resources/test_ptna_conversation.csv]


### (1-2) 訓練データのTF-IDFベクターを作成

In [3]:
X, y, vectorizer = TestTool.prepare_tf_idf_vectors(temp_path[0])

2017/03/31 PM 02:56:23 TrainingMessageFromCsv#__build_learning_training_messages count of learning data: 17443
2017/03/31 PM 02:56:23 TextArray#__init__ start
2017/03/31 PM 02:56:33 TextArray#to_vec start
2017/03/31 PM 02:56:33 TextArray#to_vec end


## (2) 動作確認

訓練データを使用し、学習 --> 予測 --> accuracy単純算出の流れで、動作確認をいたします。

### (2-1) 学習

In [4]:
from sklearn.neural_network import MLPClassifier

cls = MLPClassifier(activation='logistic', shuffle=False, random_state=0)
cls.fit(X, y)

MLPClassifier(activation='logistic', alpha=0.0001, batch_size='auto',
       beta_1=0.9, beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=0, shuffle=False,
       solver='adam', tol=0.0001, validation_fraction=0.1, verbose=False,
       warm_start=False)

### (2-2) 訓練データを使用して予測

In [5]:
y_pred = cls.predict(X) # 予測クラスが戻る
y_proba = cls.predict_proba(X) # 予測クラスの proba が戻る

### (2-3) accuracy を算出

In [6]:
''' 
    正解率を計算します。
    ---> 0.99570102029118424
'''
TestTool.accuracy_score(y, y_pred, y_proba)

0.99570102029118424

## (3) 未知のデータで予測実行

### (3-1) 未知データのTF-IDFベクターを作成

明らかにおかしい質問文と、訓練データに似た質問文で、予測を実行させてみます。

In [47]:
questions = [
    '要素技術は自然languageの機械learningですか？', # まったくfeatureが抽出されない質問文

    '人生相談をしたいのですが？', # featureが１件抽出されるな質問文「する」を含む
    '難解なプログラミング技術を調達？', # featureが１件抽出される質問文「する」を含まない

    '何か習い事をしますか？', # featureが２件抽出される質問文「する」を含む
    '何か習い事がいいですか？', # featureが２件抽出される質問文「する」を含まない
    
    '何か習い事をしたほうがいいですか？', # featureが３件抽出される質問文「する」を含む
    '何か習い事を行うほうがいいですか？', # featureが３件抽出される質問文「する」を含まない
    
    '会社を辞めたいのですが誰に相談するのがいいですか？', # featureが４件抽出される質問文「する」を含む
    '会社を辞めたいのですが誰に相談を行うのがいいですか？', # featureが４件抽出される質問文「する」を含まない

    '有給休暇を取って海外に行き旅行する意向があります。', # featureが５件抽出される質問文「する」を含む
    '有給休暇を取って海外に行きたいと思っています。', # featureが５件抽出される質問文「する」を含まない

    # 以下、訓練データの質問文と似たような feature を与えてみます。
    'ウィンドウズ７の標準バージョンは？', # Windows7社内標準のバージョンを教えてください。,3637
    'ウィンドウズのパスワードを使ってログインできない', # Windowsユーザアカウントのパスワードを入力してもログインできない。,3739
    '教育コースって何？', # 教育コースについて、教えて欲しい。,3907
    '全社の規約が知りたいですが？', # 全社規程を確認したい。,3910

    # なぜか回答不能に陥った feature はこちら（正解文と一緒にテストします）
    'アウトルックのメールの探し方がわからないんですが', # Outlook2010で、メールを探すことができません。,3442
    'Outlook2010で、メールを探すことができません。', # Outlook2010で、メールを探すことができません。,3442
]
X_test = TestTool.prepare_tf_idf_vectors_from_questions(questions, vectorizer)

2017/03/31 PM 03:28:20 TextArray#__init__ start
2017/03/31 PM 03:28:21 TextArray#to_vec start
2017/03/31 PM 03:28:21 TextArray#to_vec end


### (3-2) 未知データで予測

In [48]:
y_test_pred = cls.predict(X_test) # 予測クラスが戻る
y_test_proba = cls.predict_proba(X_test) # 予測クラスの proba が戻る

### (3-3) 予測結果を確認

概ね良好な結果を示しております。

In [49]:
import numpy

dumped_features = TestTool.get_dumped_features(X_test, vectorizer.vocabulary_)
for i, d in enumerate(dumped_features):
    c = y_test_pred[i]
    p = numpy.max(y_test_proba[i])
    if c == 0 or p < 0.5:
        print('%s ---> Unable to predict' % d)
    else:
        print('%s ---> class=%d (proba=%0.3f)' % (d, c, p))

index=0[] ---> Unable to predict
index=1[する=1.000] ---> Unable to predict
index=2[調達=1.000] ---> Unable to predict
index=3[する=0.707 何=0.707] ---> Unable to predict
index=4[いい=0.707 何=0.707] ---> Unable to predict
index=5[いい=0.577 する=0.577 何=0.577] ---> Unable to predict
index=6[いい=0.577 何=0.577 行う=0.577] ---> Unable to predict
index=7[いい=0.500 する=0.500 会社=0.500 誰=0.500] ---> Unable to predict
index=8[いい=0.500 会社=0.500 行う=0.500 誰=0.500] ---> Unable to predict
index=9[する=0.447 休暇=0.447 取る=0.447 海外=0.447 行く=0.447] ---> Unable to predict
index=10[休暇=0.447 取る=0.447 思う=0.447 海外=0.447 行く=0.447] ---> Unable to predict
index=11[ウィンドウズ=0.500 バージョン=0.500 標準=0.500 ７=0.500] ---> class=3637 (proba=0.989)
index=12[できる=0.408 ない=0.408 ウィンドウズ=0.408 パスワード=0.408 ログイン=0.408 使う=0.408] ---> class=3739 (proba=0.963)
index=13[コース=0.577 何=0.577 教育=0.577] ---> class=3907 (proba=0.844)
index=14[全社=0.577 知る=0.577 規約=0.577] ---> class=3910 (proba=0.978)
index=15[ない=0.378 わかる=0.378 アウト=0.378 メール=0.378 ルック=0.378 探す=0